- This file is used to scrap all the main_categories & sub categories from the arxiv url and save the result into delta table

In [1]:
from pyspark.sql import SparkSession
import pyspark
import requests
from bs4 import BeautifulSoup
from delta import DeltaTable, configure_spark_with_delta_pip

builder = pyspark.sql.SparkSession.builder.appName("Scraping_Category") \
    .master('yarn')\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config('spark.executor.instances', '12') \
    .config("spark.executor.memory", "1g") \
    
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b3b1a1c0-5916-4db6-98af-f6e73caac0e1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 204ms :: artifacts dl 10ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

23/04/25 14:10:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/25 14:10:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/25 14:10:31 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
23/04/25 14:10:41 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-core_2.12-2.3.0.jar added multiple times to distributed cache.
23/04/25 14:10:41 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/io.delta_delta-storage-2.3.0.jar added multiple times to distributed cache.
23/04/25 14:10:41 WARN Client: Same path resource file:///home/ubuntu/.ivy2/jars/org.antlr_antlr4-runtime-4.8.jar added multiple times to distributed cache.


In [7]:
# the content of the category & subcateory in this url
url = 'https://arxiv.org/category_taxonomy'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

# Find the main categories section of the page
main_category = soup.find_all('h2', {'class': 'accordion-head'})

# collect all the main categories int a dictionary
category_dict = {category.text: [] for category in main_category}

# Add all sub categories into list
Sub_categories_list = []
for div in soup.find_all('div', class_='columns divided'):
    h4 = div.find('h4')
    span = h4.find('span')
    if span:
        span = span.text.strip('()')
    sub_category = h4.text.strip().split()[0]
    if sub_category == "Category":
        continue
    Sub_categories_list.append((sub_category, span))

# Add all sub categories to related main_category
for cat in Sub_categories_list:
    if cat[0].split('.')[0] == 'cs':
        category_dict.get('Computer Science').append(cat)
    elif cat[0].split('.')[0] == 'econ':
        category_dict.get('Economics').append(cat)
    elif cat[0].split('.')[0] == 'eess':
        category_dict.get('Electrical Engineering and Systems Science').append(cat)
    elif cat[0].split('.')[0] == 'math':
        category_dict.get('Mathematics').append(cat)
    elif cat[0].split('.')[0] == 'q-bio':
        category_dict.get('Quantitative Biology').append(cat) 
    elif cat[0].split('.')[0] == 'q-fin':
        category_dict.get('Quantitative Finance').append(cat)       
    elif cat[0].split('.')[0] == 'stat':
        category_dict.get('Statistics').append(cat)
    else:
        category_dict.get('Physics').append(cat)

# prepare creating a new dataframe for the category from a list of tuples which contains the candidates values
category_tuples = []
for main, subList in category_dict.items():
    print(main, subList)
    for sub in subList:    
        category_tuples.append((main, sub[0], sub[1]))


    


Computer Science [('cs.AI', 'Artificial Intelligence'), ('cs.AR', 'Hardware Architecture'), ('cs.CC', 'Computational Complexity'), ('cs.CE', 'Computational Engineering, Finance, and Science'), ('cs.CG', 'Computational Geometry'), ('cs.CL', 'Computation and Language'), ('cs.CR', 'Cryptography and Security'), ('cs.CV', 'Computer Vision and Pattern Recognition'), ('cs.CY', 'Computers and Society'), ('cs.DB', 'Databases'), ('cs.DC', 'Distributed, Parallel, and Cluster Computing'), ('cs.DL', 'Digital Libraries'), ('cs.DM', 'Discrete Mathematics'), ('cs.DS', 'Data Structures and Algorithms'), ('cs.ET', 'Emerging Technologies'), ('cs.FL', 'Formal Languages and Automata Theory'), ('cs.GL', 'General Literature'), ('cs.GR', 'Graphics'), ('cs.GT', 'Computer Science and Game Theory'), ('cs.HC', 'Human-Computer Interaction'), ('cs.IR', 'Information Retrieval'), ('cs.IT', 'Information Theory'), ('cs.LG', 'Machine Learning'), ('cs.LO', 'Logic in Computer Science'), ('cs.MA', 'Multiagent Systems'), ('

In [8]:
category_df = spark.createDataFrame(category_tuples, ["main_category", "sub_category", "description"])

In [12]:
category_df.show(3)

+----------------+------------+--------------------+
|   main_category|sub_category|         description|
+----------------+------------+--------------------+
|Computer Science|       cs.AI|Artificial Intell...|
|Computer Science|       cs.AR|Hardware Architec...|
|Computer Science|       cs.CC|Computational Com...|
+----------------+------------+--------------------+
only showing top 3 rows



- update the main category that = 'Electrical Engineering and Systems Science' to take short name "Electrical Engineering"

In [13]:
from pyspark.sql.functions import when, col

category_df = category_df.withColumn("main_category",
                   when(category_df.main_category == "Electrical Engineering and Systems Science", "Electrical Engineering")
                   .otherwise(category_df.main_category))
    

In [17]:
category_df.filter(col("main_category")== "Electrical Engineering").show(3, False)

+----------------------+------------+---------------------------+
|main_category         |sub_category|description                |
+----------------------+------------+---------------------------+
|Electrical Engineering|eess.AS     |Audio and Speech Processing|
|Electrical Engineering|eess.IV     |Image and Video Processing |
|Electrical Engineering|eess.SP     |Signal Processing          |
+----------------------+------------+---------------------------+
only showing top 3 rows



In [18]:
# save category data into delta table
#=====================================================================
delta_category_file = "hdfs:///Dat500_Group09/spark_result/category"
if not DeltaTable.isDeltaTable(spark, delta_category_file):
    print("save category data")      
    category_df.write.format("delta").save(delta_category_file)


save category data


23/04/25 14:29:54 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [19]:
# To check that dataframe is saved
df = spark.read.format("delta").load(delta_category_file)
df.count()

155

In [20]:
df.show()

+----------------+------------+--------------------+
|   main_category|sub_category|         description|
+----------------+------------+--------------------+
|     Mathematics|     math.OA|   Operator Algebras|
|     Mathematics|     math.OC|Optimization and ...|
|     Mathematics|     math.PR|         Probability|
|     Mathematics|     math.QA|     Quantum Algebra|
|     Mathematics|     math.RA|  Rings and Algebras|
|     Mathematics|     math.RT|Representation Th...|
|     Mathematics|     math.SG| Symplectic Geometry|
|     Mathematics|     math.SP|     Spectral Theory|
|     Mathematics|     math.ST|   Statistics Theory|
|         Physics| astro-ph.CO|Cosmology and Non...|
|         Physics| astro-ph.EP|Earth and Planeta...|
|         Physics| astro-ph.GA|Astrophysics of G...|
|         Physics| astro-ph.HE|High Energy Astro...|
|         Physics| astro-ph.IM|Instrumentation a...|
|Computer Science|       cs.NE|Neural and Evolut...|
|Computer Science|       cs.NI|Networking and 